## Criteria evaluator

### Instalacja bibliotek

In [86]:
! pip install -U langsmith langchain langchain-core langchain-text-splitters

In [87]:
from dotenv import load_dotenv
load_dotenv()

True

### Prosta ewaluacja za pomocą kryteriów
Criteria evaluator w LangChain służy do oceniania wyników modeli LLM według zadanego kryterium, np. correctness (poprawność).

In [88]:
from langchain_classic.evaluation import load_evaluator, Criteria, EvaluatorType
import json

# 1) Użyjemy ewaluatora z referencją (labeled_criteria)
evaluator = load_evaluator("labeled_criteria", criteria="correctness")

# 2) Porównujemy odpowiedź modelu z referencją
result = evaluator.evaluate_strings(
    prediction="2 + 2 = 4",
    input="Policz 2 + 2",
    reference="4",
)

print(json.dumps(result, indent=4))

{
    "reasoning": "The criterion for this task is the correctness of the submitted answer. The input asks to calculate 2 + 2. The submitted answer is 2 + 2 = 4. Comparing this with the reference answer, which is 4, it is clear that the submitted answer is correct. Therefore, the submission meets the criterion of correctness.\n\nY",
    "value": "Y",
    "score": 1
}


### Import bibliotek i konfiguracja LangSmith

In [89]:
import os
from langsmith import Client
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
load_dotenv()

# Włącz śledzenie LangSmith (wymaga konta LangSmith):
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "kurs-demo"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
# os.environ["LANGSMITH_API_KEY"] = "<TWÓJ_KLUCZ>" #załączony w .env

### Wygenerowanie odpowiedzi LLM

In [90]:
client = Client()
dataset_inputs = [
    "Why people don't have 3 legs?",
    "Why people are not flying?",
]

llm_test= ChatOpenAI(model="gpt-3.5-turbo", temperature=0.1,max_tokens=256)
llm_gen = ChatOpenAI(model="gpt-4o", temperature=0.1,max_tokens=256)

dataset_outputs = [
    {"result": llm_test.invoke(dataset_inputs[0])},
    {"result": llm_test.invoke(dataset_inputs[1])},
]
print(dataset_outputs)

[{'result': AIMessage(content='Humans typically have two legs because that is the natural and most efficient way for our bodies to move and function. Having three legs would likely be more cumbersome and less practical for everyday activities. Additionally, evolution has shaped humans to have two legs for millions of years, so there has been no need for a third leg to develop.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 65, 'prompt_tokens': 16, 'total_tokens': 81, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CaFZvjlnYDFwlg2cfG6wf1aFfqp6E', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--7760d2af-f961-4e22-9236-7cfdf0375820-0', usage_meta

### Custom evaluator

In [91]:
from langchain_classic.smith import RunEvalConfig, run_on_dataset
from langchain_classic.evaluation import Criteria
from langchain_classic.evaluation.schema import EvaluatorType
from langsmith.evaluation import EvaluationResult, run_evaluator

In [92]:
@run_evaluator
def custom_evaluator(run) -> EvaluationResult:
    """
    checks if output contains specific word
    :param run:
    :return: int
    """
    generated = str(run).lower()
    if 'human' in generated:
        score = 1
    else:
        score = 0
    return EvaluationResult(key="result", score=score)

eval_config = RunEvalConfig(
    custom_evaluators=[custom_evaluator],
    evaluators=[
        EvaluatorType.CRITERIA,
        EvaluatorType.QA,         # bezpośrednio oceń odpowiedź jako "poprawną" lub "niepoprawną" na podstawie odpowiedzi referencyjnej
        EvaluatorType.CONTEXT_QA, # użyj podanego kontekstu odniesienia w celu ustalenia poprawności
        EvaluatorType.COT_QA,     # chain of thought "reasoning"
        RunEvalConfig.Criteria(criteria=Criteria.INSENSITIVITY),
        RunEvalConfig.Criteria(criteria=Criteria.RELEVANCE),
        RunEvalConfig.Criteria(criteria=Criteria.HELPFULNESS),
        RunEvalConfig.Criteria(criteria=Criteria.MALICIOUSNESS),
        RunEvalConfig.Criteria(criteria=Criteria.HARMFULNESS),
        RunEvalConfig.Criteria(criteria=Criteria.COHERENCE),
        RunEvalConfig.Criteria(criteria=Criteria.CONCISENESS),
        RunEvalConfig.Criteria(criteria=Criteria.MISOGYNY),
        RunEvalConfig.Criteria(criteria=Criteria.CRIMINALITY),
        RunEvalConfig.Criteria(criteria=Criteria.CONTROVERSIALITY),
        RunEvalConfig.Criteria( # własne zdefiniowane kryteria, które dotyczą problemu wystepujacego w generowanych odpowiedziach
            criteria={
                "valuation": "Do texts contain valuation of subject, like glorifying some characteristic or judging someone?"
                "Respond Y if they do, N if they're entirely objective and stick to the facts without additions."
            }
        )
    ],
)

### Uruchomienie i integracja z LangSmith

In [93]:
import uuid

dataset_name = "existential questions run:" + uuid.uuid4().__str__() #wymagana zmiana parametru przy każdym uruchomieniu

dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="evaluate LLM output",
)
client.create_examples(
    inputs=[{"question": q} for q in dataset_inputs],
    outputs=dataset_outputs,
    dataset_id=dataset.id,
)

{'example_ids': ['9654a8ee-130b-4a71-b9f1-62552313fa36',
  'f0591650-e847-4eb6-a08c-43233beaf7f8'],
 'count': 2}

In [94]:
eval_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

def construct_chain():
    return llm_gen

scores = run_on_dataset(
    client=client,
    dataset_name=dataset_name,
    llm_or_chain_factory=construct_chain,
    evaluation=eval_config,
    project_name=dataset_name,
    verbose=True,
)
print(scores)


View the evaluation results for project 'existential questions run:15be034d-9d5e-4b46-8d31-861a1c301b7e' at:
https://smith.langchain.com/o/3e1f981e-76ef-5491-9a42-e33f3bdfeba4/datasets/6b857296-ae02-461d-94a1-dd8552280080/compare?selectedSessions=a6ef8ce1-975b-48c9-8583-32d38d6b5c09

View all tests for Dataset existential questions run:15be034d-9d5e-4b46-8d31-861a1c301b7e at:
https://smith.langchain.com/o/3e1f981e-76ef-5491-9a42-e33f3bdfeba4/datasets/6b857296-ae02-461d-94a1-dd8552280080
[------------------------------------------------->] 2/2


,feedback.helpfulness,feedback.correctness,feedback.Contextual Accuracy,feedback.COT Contextual Accuracy,feedback.insensitivity,feedback.relevance,feedback.maliciousness,feedback.harmfulness,feedback.coherence,feedback.conciseness,feedback.misogyny,feedback.criminality,feedback.controversiality,feedback.valuation,feedback.result,error,execution_time,run_id
count,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,0,2.000000,2
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,2
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,614d55cf-39d2-4dec-afe7-6aed37b6a76b
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
mean,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,3.941025,NaN
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.947462,NaN
min,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,3.271068,NaN
25%,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,3.606046,NaN
50%,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,3.941025,NaN
75%,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,4.276003,NaN


{'project_name': 'existential questions run:15be034d-9d5e-4b46-8d31-861a1c301b7e', 'results': {'9654a8ee-130b-4a71-b9f1-62552313fa36': {'input': {'question': "Why people don't have 3 legs?"}, 'feedback': [EvaluationResult(key='helpfulness', score=1, value='Y', comment='The criterion for this task is "helpfulness". \n\nThe AI\'s response to the question "Why people don\'t have 3 legs?" is detailed and informative. It provides a comprehensive explanation of why humans have two legs, citing reasons such as bipedalism, balance and coordination, evolutionary path, and energy efficiency. \n\nThe response is helpful as it provides a clear and understandable answer to the question. It is insightful as it delves into the evolutionary reasons behind the human body structure. The response is also appropriate as it directly addresses the question asked.\n\nTherefore, the submission meets the criterion of being helpful, insightful, and appropriate.\n\nY', correction=None, evaluator_info={'__run': R